In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.2-1b-preview",
    temperature=0,
    groq_api_key="enter your api key")

response = llm.invoke('Who was the first man to walk on moon')
print(response.content)

The first man to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first person to set foot on the moon.

Armstrong was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the moon's surface. Michael Collins remained in orbit around the moon in the command module Columbia.

Neil Armstrong's historic moonwalk lasted about two and a half hours, and he collected samples and conducted experiments during his time on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/97411123313025734-software-engineer-iii-corpeng?location=India")
page_data = loader.load().pop().page_content
print(page_data)

Software Engineer III, CorpEng — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results418  jobs matchedAccount Executive, Mid-Market Sales, Google Customer SolutionsMumbai, Maharashtra, India; Gurgaon, Haryana, IndiaEngineering Manager, NetworkingBengaluru, Karnataka, IndiaAdministrative Business Partner, Cloud AppEcoHyderabad, Telangana, India; Bangalore, Karnataka, IndiaBusiness Intelligence AnalystHyderabad, Telangana, IndiaNetwork Implementation Engineer, Data Center Networking DeliveryMumbai, Maharashtra, IndiaSoftware Engineer III, CorpEngHyderabad, Telangana, IndiaStrategic Partner Manager, Influencer Marketing Agencies, YouTubeGurugram, Haryana, India; Mumbai, Maharashtra, IndiaHR Compliance ManagerBenga

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPPED TEXT FROM WEBPAGE:
    {page_data}
    ### INSTRUCTION:

    The scrapped text is from career page of a website.
    Your job is extract the job posting and return them in JSON format containing the 
    following keys: 'role', 'experience', 'skills', and 'description'.

    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

In [7]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
    "role": "Software Engineer III, CorpEng",
    "experience": {
        "years": 2,
        "experience": [
            {
                "degree": "Bachelor’s degree or equivalent practical experience",
                "language": "2 years of experience with software development in one or more programming languages, or 1 year of experience with an advanced degree"
            },
            {
                "degree": "Master’s degree or PhD in Computer Science or related technical fields",
                "language": "Preferred qualifications: Master's degree or PhD in Computer Science or related technical fields"
            }
        ]
    },
    "skills": [
        "2 years of experience with data structures or algorithms",
        "experience developing accessible technologies",
        "experience with information retrieval, distributed computing, large-scale system design, networking and data storage, security, artificial intelligence, natural language processing, U

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III, CorpEng',
 'experience': {'years': 2,
  'experience': [{'degree': 'Bachelor’s degree or equivalent practical experience',
    'language': '2 years of experience with software development in one or more programming languages, or 1 year of experience with an advanced degree'},
   {'degree': 'Master’s degree or PhD in Computer Science or related technical fields',
    'language': "Preferred qualifications: Master's degree or PhD in Computer Science or related technical fields"}]},
 'skills': ['2 years of experience with data structures or algorithms',
  'experience developing accessible technologies',
  'experience with information retrieval, distributed computing, large-scale system design, networking and data storage, security, artificial intelligence, natural language processing, UI design and mobile'],
 'description': "Write product or system development code. Participate in, or lead design reviews with peers and stakeholders to decide amongst availabl

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={"links":row['Links']},
                       ids=[str(uuid.uuid4())]
                       )

In [14]:
job = json_res
job['skills']

['2 years of experience with data structures or algorithms',
 'experience developing accessible technologies',
 'experience with information retrieval, distributed computing, large-scale system design, networking and data storage, security, artificial intelligence, natural language processing, UI design and mobile']

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Alex, a business development executive at InnovaX. InnovaX is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of InnovaX 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase InnovaX's portfolio: {link_list}
        Remember you are Alex, BDE at InnovaX. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": 'Links'})
print(res.content)

Subject: Enhance Your Business Processes with InnovaX's Expertise

Dear [Client],

As a Business Development Executive at InnovaX, I've had the pleasure of working with numerous enterprises that have benefited from our tailored solutions. Our team of experts has a proven track record of empowering businesses to scale, optimize, and reduce costs through the seamless integration of business processes.

At InnovaX, we specialize in developing innovative AI & software consulting solutions that cater to the unique needs of each client. Our capabilities include:

- Developing accessible technologies that cater to diverse user needs
- Designing and implementing large-scale systems that drive efficiency and scalability
- Integrating data structures and algorithms to drive business growth
- Providing expertise in information retrieval, distributed computing, and data storage
- Offering security, artificial intelligence, natural language processing, UI design, and mobile solutions

Our team has 